# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'05-18-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'05-18-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-05-19 04:20:42,33.93911,67.709953,63819,2751,55118.0,5950.0,Afghanistan,163.939686,4.310628
1,NaN,NaN,NaN,Albania,2021-05-19 04:20:42,41.15330,20.168300,132071,2436,126405.0,3230.0,Albania,4589.304330,1.844462
2,NaN,NaN,NaN,Algeria,2021-05-19 04:20:42,28.03390,1.659600,125693,3388,87609.0,34696.0,Algeria,286.636284,2.695456
3,NaN,NaN,NaN,Andorra,2021-05-19 04:20:42,42.50630,1.521800,13569,127,13234.0,208.0,Andorra,17561.638517,0.935957
4,NaN,NaN,NaN,Angola,2021-05-19 04:20:42,-11.20270,17.873900,31045,685,26013.0,4347.0,Angola,94.458549,2.206474


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,61842,62063,62403,62718,63045,63355,63412,63484,63598,63819
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131723,131753,131803,131845,131890,131939,131978,132015,132032,132071
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,124104,124288,124483,124682,124889,125059,125194,125311,125485,125693


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2686,2698,2710,2713,2721,2730,2733,2742,2745,2751
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2412,2416,2420,2423,2426,2427,2429,2432,2435,2436
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3328,3335,3343,3350,3355,3360,3366,3374,3381,3388


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54222,54382,54503,54534,54619,54634,54663,54686,55010,55118
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,117089,118041,119061,120072,121122,122105,123081,124312,125419,126405
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,86420,86554,86703,86857,87003,87137,87251,87359,87476,87609


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6920,6920,6926,6928,6938,6971,7001,7005,7010,7015
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21131,21135,21154,21170,21191,21290,21392,21411,21422,21444
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2308,2308,2310,2314,2317,2319,2320,2320,2320,2322


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,108,108,108,108,108,108,108,108,108,108
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,308,308,308,309,309,309,310,310,310,310
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,57,57,57,56,56,56,56,56,56,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-05-19 04:20:42,32.539527,-86.644082,7015,108,NaN,NaN,"Autauga, Alabama, US",12556.158156,1.539558
689,1075.0,Lamar,Alabama,US,2021-05-19 04:20:42,33.779950,-88.096680,1434,36,NaN,NaN,"Lamar, Alabama, US",10387.540746,2.510460
690,1077.0,Lauderdale,Alabama,US,2021-05-19 04:20:42,34.901719,-87.656247,9474,243,NaN,NaN,"Lauderdale, Alabama, US",10216.868509,2.564915


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32996786,587203,0.0
India,25228996,278719,21596512.0
Brazil,15732836,439050,13957814.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32996786,587203,0.0,32409583.0,2021-05-19 04:20:42,37.950547,-91.419547
India,25228996,278719,21596512.0,3353765.0,2021-05-19 04:20:42,23.088275,81.806127
Brazil,15732836,439050,13957814.0,1335972.0,2021-05-19 04:20:42,-12.669522,-48.480493
France,5959593,108201,381251.0,5470141.0,2021-05-19 04:20:42,6.430808,-34.730285
Turkey,5139485,45186,4971245.0,123054.0,2021-05-19 04:20:42,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3770695,62711,0.0
Texas,2934592,51012,0.0
Florida,2296785,36227,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3770695,62711,0.0,3707984.0,2021-05-19 04:20:42,37.843962,-120.728594
Texas,2934592,51012,0.0,2883580.0,2021-05-19 04:20:42,31.660643,-98.653069
Florida,2296785,36227,0.0,2260558.0,2021-05-19 04:20:42,28.940755,-82.700744
New York,2088361,53020,0.0,2035341.0,2021-05-19 04:20:42,42.544151,-75.474183
Illinois,1368622,24830,0.0,1343792.0,2021-05-19 04:20:42,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1237681,24123,0.0
Illinois,Cook,547922,10161,0.0
Arizona,Maricopa,545468,9963,0.0
Florida,Miami-Dade,495921,6333,0.0
Texas,Harris,397046,6345,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1237681,24123,0.0,1213558.0,2021-05-19 04:20:42,34.308284,-118.228241,6037.0
Illinois,Cook,547922,10161,0.0,537761.0,2021-05-19 04:20:42,41.841448,-87.816588,17031.0
Arizona,Maricopa,545468,9963,0.0,535505.0,2021-05-19 04:20:42,33.348359,-112.491815,4013.0
Florida,Miami-Dade,495921,6333,0.0,489588.0,2021-05-19 04:20:42,25.611236,-80.551706,12086.0
Texas,Harris,397046,6345,0.0,390701.0,2021-05-19 04:20:42,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,63484,132015,125311,13510,30637,1241,3307285,220860,29978,637097,...,4466589,240512,96670,4,215301,4212,303827,6543,92436,38560
2021-05-17,63598,132032,125485,13555,30787,1251,3335965,220927,29983,637573,...,4468582,242906,96893,4,216415,4359,304074,6568,92460,38572
2021-05-18,63819,132071,125693,13569,31045,1251,3371508,221139,29992,638155,...,4466220,246026,97068,4,217603,4512,304074,6586,92520,38595


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,2742,2432,3374,127,659,33,70522,4323,910,10474,...,127941,3459,668,1,2396,37,3428,1289,1260,1582
2021-05-17,2745,2435,3381,127,677,42,71027,4333,910,10480,...,127946,3521,670,1,2411,37,3437,1294,1261,1582
2021-05-18,2751,2436,3388,127,685,42,71771,4346,910,10499,...,127953,3578,672,1,2428,37,3437,1297,1263,1583


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,54686,124312,87359,13155,25715,1179,2952599,208454,23494,616148,...,15258,209338,92111,3,199202,2668,292523,3023,90816,36329
2021-05-17,55010,125419,87476,13211,25995,1182,2973267,208899,23499,617307,...,15292,212051,92303,3,200381,2668,293808,3042,90862,36349
2021-05-18,55118,126405,87609,13234,26013,1182,2992325,209222,23511,618798,...,15319,214578,92582,3,201646,2687,293808,3072,90892,36371


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6920,6920,6926,6928,6938,6971,7001,7005,7010,7015
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1425,1425,1425,1427,1429,1430,1432,1433,1433,1434
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9424,9428,9427,9437,9441,9449,9457,9460,9465,9474
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3002,3002,3003,3004,3007,3022,3026,3030,3036,3041
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15884,15886,15882,15887,15892,15946,15987,15993,15997,16005


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,540083,69464,0,872978,338782,3768805,531700,344977,107007,49,...,857055,2929050,402567,23883,3267,670184,422665,158468,669668,59079
2021-05-17,540267,69600,0,873446,338849,3769827,532389,345639,107089,49,...,858355,2931500,402731,23912,3267,670456,423382,158643,669804,59222
2021-05-18,540603,69657,0,874065,339162,3770695,533119,345720,107175,49,...,858804,2934592,402988,23945,3288,670834,424050,158888,670362,59321


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-05-16        7005   21411    2320  2647   6773    1232   2191   14574   
2021-05-17        7010   21422    2320  2648   6776    1233   2190   14577   
2021-05-18        7015   21444    2322  2651   6794    1233   2190   14580   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-16         3596     1848  ...    2758    630     3179      790   
2021-05-17         3598     1848  ...    2769    633     3182      791   
2021-05-18         3606     1849  ...    2779    633     3184      791   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-16           4353  3765  2239          0      918    651  
2021-05-17           4377  3769  2244          0      917    651  
2021-05-18           4384  3772  2249          0      917    651  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,11038,353,0,17459,5794,62690,6450,8173,1651,0,...,12312,50941,2258,252,27,11018,5622,2761,7712,712
2021-05-17,11038,364,0,17466,5796,62699,6450,8194,1651,0,...,12327,50972,2259,252,27,11029,5640,2762,7712,712
2021-05-18,11043,368,0,17480,5805,62711,6456,8198,1651,0,...,12339,51012,2271,254,27,11042,5653,2763,7726,713


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-05-16         108     310      56   64    139      42     69     319   
2021-05-17         108     310      56   64    139      42     69     319   
2021-05-18         108     310      56   64    139      42     69     319   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-16          123       45  ...      30     11       31        7   
2021-05-17          123       45  ...      30     11       31        7   
2021-05-18          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-16             38     9    13          0       26      5  
2021-05-17             38     9    13          0       26      5  
2021-05-18             38     9    13          1       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.001135,0.000280,0.000935,0.000000,0.009323,0.000000,0.004968,0.000593,0.000100,0.001057,...,0.000431,0.010219,0.003134,0.0,0.010812,0.024319,0.000557,0.000765,0.000292,0.000156
2021-05-17,0.001796,0.000129,0.001389,0.003331,0.004896,0.008058,0.008672,0.000303,0.000167,0.000747,...,0.000446,0.009954,0.002307,0.0,0.005174,0.034900,0.000813,0.003821,0.000260,0.000311
2021-05-18,0.003475,0.000295,0.001658,0.001033,0.008380,0.000000,0.010654,0.000960,0.000300,0.000913,...,-0.000529,0.012844,0.001806,0.0,0.005489,0.035100,0.000000,0.002741,0.000649,0.000596


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.003293,0.001235,0.002377,0.0,0.006107,0.000000,0.003829,0.002086,0.0,0.000382,...,0.000031,0.011699,0.001499,0.0,0.012680,0.027778,0.001461,0.003113,0.000000,0.000000
2021-05-17,0.001094,0.001234,0.002075,0.0,0.027314,0.272727,0.007161,0.002313,0.0,0.000573,...,0.000039,0.017924,0.002994,0.0,0.006260,0.000000,0.002625,0.003879,0.000794,0.000000
2021-05-18,0.002186,0.000411,0.002070,0.0,0.011817,0.000000,0.010475,0.003000,0.0,0.001813,...,0.000055,0.016189,0.002985,0.0,0.007051,0.000000,0.000000,0.002318,0.001586,0.000632


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.000421,0.010002,0.001238,0.000000,0.000467,0.000000,0.006358,0.001701,0.000043,0.002188,...,0.000262,0.012909,0.003049,0.0,0.012416,0.000000,0.004574,0.002986,0.000430,0.000303
2021-05-17,0.005925,0.008905,0.001339,0.004257,0.010889,0.002545,0.007000,0.002135,0.000213,0.001881,...,0.002228,0.012960,0.002084,0.0,0.005919,0.000000,0.004393,0.006285,0.000507,0.000551
2021-05-18,0.001963,0.007862,0.001520,0.001741,0.000692,0.000000,0.006410,0.001546,0.000511,0.002415,...,0.001766,0.011917,0.003023,0.0,0.006313,0.007121,0.000000,0.009862,0.000330,0.000605


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,0.000471,0.000000,NaN,0.000552,0.000280,0.000191,0.001186,0.000000,0.001254,0.0,...,0.000000,0.000226,0.000602,0.001510,0.000000,0.000418,0.000000,0.001504,0.000360,0.000000
2021-05-17,0.000341,0.001958,NaN,0.000536,0.000198,0.000271,0.001296,0.001919,0.000766,0.0,...,0.001517,0.000836,0.000407,0.001214,0.000000,0.000406,0.001696,0.001104,0.000203,0.002420
2021-05-18,0.000622,0.000819,NaN,0.000709,0.000924,0.000230,0.001371,0.000234,0.000803,0.0,...,0.000523,0.001055,0.000638,0.001380,0.006428,0.000564,0.001578,0.001544,0.000833,0.001672


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-16      0.000571  0.000888  0.000000  0.000756  0.000295  0.000000   
2021-05-17      0.000714  0.000514  0.000000  0.000378  0.000443  0.000812   
2021-05-18      0.000713  0.001027  0.000862  0.001133  0.002656  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-16      0.000457  0.000343  0.001671  0.001083  ...  0.000000   
2021-05-17     -0.000456  0.000206  0.000556  0.000000  ...  0.003988   
2021-05-18      0.000000  0.000206  0.002223  0.000541  ...  0.003611   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-16      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-05-17      0.004762  0.000944  0.001266   0.005513  0.001062  0.002233   
2021-05-18      0.000000  0.000629  0.000000   0.001599  0.000796  0.002228   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-05-16            NaN  0.000000    0.0  
2021-05-17            NaN -0.001089    0.0  
2021-05-18            NaN  0.000000    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,0.000000,0.000000,NaN,0.000000,0.000173,0.000543,0.001242,0.000000,0.0,NaN,...,0.000000,0.000020,0.000000,0.000000,0.0,0.000908,0.000000,0.001451,0.000000,0.000000
2021-05-17,0.000000,0.031161,NaN,0.000401,0.000345,0.000144,0.000000,0.002569,0.0,NaN,...,0.001218,0.000609,0.000443,0.000000,0.0,0.000998,0.003202,0.000362,0.000000,0.000000
2021-05-18,0.000453,0.010989,NaN,0.000802,0.001553,0.000191,0.000930,0.000488,0.0,NaN,...,0.000973,0.000785,0.005312,0.007937,0.0,0.001179,0.002305,0.000362,0.001815,0.001404


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-05-16         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-05-17         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-05-18         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-16          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-05-17          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-05-18          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-16            0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-17            0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-18            0.0   0.0   0.0        inf      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.002029,0.000304,0.001110,0.000460,0.009922,0.000300,0.006148,0.000904,0.000194,0.001154,...,0.000463,0.011668,0.003080,4.967054e-09,0.006647,0.037095,0.000701,0.001423,0.000484,0.000410
2021-05-17,0.001912,0.000216,0.001249,0.001895,0.007409,0.004179,0.007410,0.000604,0.000180,0.000950,...,0.000454,0.010811,0.002693,2.483527e-09,0.005911,0.035997,0.000757,0.002622,0.000372,0.000361
2021-05-18,0.002694,0.000256,0.001454,0.001464,0.007895,0.002089,0.009032,0.000782,0.000240,0.000932,...,-0.000037,0.011828,0.002250,1.241763e-09,0.005700,0.035549,0.000378,0.002681,0.000510,0.000479


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.002676,0.001036,0.002050,5.046402e-07,0.005809,0.007813,0.005268,0.002562,3.201742e-14,0.000873,...,0.000058,0.013606,0.000997,0.0,0.007917,0.021032,0.002560,0.002366,0.000089,0.000150
2021-05-17,0.001885,0.001135,0.002063,2.523201e-07,0.016562,0.140270,0.006214,0.002438,1.600871e-14,0.000723,...,0.000049,0.015765,0.001996,0.0,0.007089,0.010516,0.002593,0.003122,0.000441,0.000075
2021-05-18,0.002035,0.000773,0.002066,1.261601e-07,0.014189,0.070135,0.008345,0.002719,8.004354e-15,0.001268,...,0.000052,0.015977,0.002490,0.0,0.007070,0.005258,0.001296,0.002720,0.001014,0.000353


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-16,0.000568,0.009086,0.001351,0.000661,0.002268,0.000740,0.006754,0.002398,0.000139,0.002652,...,0.001207,0.012779,0.003139,1.192093e-07,0.007855,0.001828,0.004482,0.002257,0.000590,0.000403
2021-05-17,0.003247,0.008996,0.001345,0.002459,0.006578,0.001642,0.006877,0.002266,0.000176,0.002266,...,0.001718,0.012870,0.002612,5.960464e-08,0.006887,0.000914,0.004437,0.004271,0.000548,0.000477
2021-05-18,0.002605,0.008429,0.001433,0.002100,0.003635,0.000821,0.006643,0.001906,0.000343,0.002341,...,0.001742,0.012393,0.002817,2.980232e-08,0.006600,0.004018,0.002219,0.007067,0.000439,0.000541


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,0.002496,0.000364,NaN,0.000614,0.000454,0.000297,0.001783,0.000275,0.001297,1.964464e-129,...,0.000204,0.000418,0.000725,0.002022,0.000641,0.000663,0.001624,0.001760,0.000631,0.000357
2021-05-17,0.001419,0.001161,NaN,0.000575,0.000326,0.000284,0.001539,0.001097,0.001032,9.822318e-130,...,0.000860,0.000627,0.000566,0.001618,0.000320,0.000535,0.001660,0.001432,0.000417,0.001389
2021-05-18,0.001020,0.000990,NaN,0.000642,0.000625,0.000257,0.001455,0.000666,0.000917,4.911159e-130,...,0.000692,0.000841,0.000602,0.001499,0.003374,0.000549,0.001619,0.001488,0.000625,0.001530


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-16      0.002071  0.002333  0.000367  0.002645  0.002159  0.000585   
2021-05-17      0.001392  0.001423  0.000183  0.001511  0.001301  0.000698   
2021-05-18      0.001053  0.001225  0.000523  0.001322  0.001979  0.000349   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-16      0.002480  0.000738  0.001688  0.000547  ...  0.000571   
2021-05-17      0.001012  0.000472  0.001122  0.000273  ...  0.002280   
2021-05-18      0.000506  0.000339  0.001673  0.000407  ...  0.002946   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-16      0.000276  0.000252  0.000141   0.000804  0.000171  0.000292   
2021-05-17      0.002519  0.000598  0.000703   0.003159  0.000617  0.001262   
2021-05-18      0.001260  0.000613  0.000352   0.002379  0.000706  0.001745   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-05-16           -1.0  0.000335  1.028203e-07  
2021-05-17           -1.0 -0.000377  5.141013e-08  
2021-05-18           -1.0 -0.000189  2.570506e-08  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-16,0.000651,0.000269,NaN,0.000290,0.000492,0.000654,0.001581,0.000171,0.000441,NaN,...,0.000163,0.000480,0.000711,0.000252,1.432804e-10,0.001418,0.000217,0.001319,0.000457,0.000045
2021-05-17,0.000325,0.015715,NaN,0.000345,0.000419,0.000399,0.000791,0.001370,0.000221,NaN,...,0.000690,0.000544,0.000577,0.000126,7.164020e-11,0.001208,0.001709,0.000840,0.000229,0.000022
2021-05-18,0.000389,0.013352,NaN,0.000573,0.000986,0.000295,0.000860,0.000929,0.000110,NaN,...,0.000832,0.000665,0.002944,0.004031,3.582010e-11,0.001193,0.002007,0.000601,0.001022,0.000713


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                          \
Admin2               Autauga   Baldwin   Barbour      Bibb    Blount   
2021-05-16      2.281856e-06  0.000918 -0.000540  0.000016  0.001834   
2021-05-17      1.140928e-06  0.000459 -0.000270  0.000008  0.000917   
2021-05-18      5.704640e-07  0.000229 -0.000135  0.000004  0.000458   

Province_State                                                                \
Admin2           Bullock        Butler   Calhoun      Chambers      Cherokee   
2021-05-16      0.006098  1.768742e-09  0.001390  1.878806e-07  1.695006e-10   
2021-05-17      0.003049  8.843710e-10  0.000695  9.394031e-08  8.475029e-11   
2021-05-18      0.001524  4.421855e-10  0.000347  4.697016e-08  4.237515e-11   

Province_State  ...       Wyoming                                            \
Admin2          ...          Park        Platte      Sheridan      Sublette   
2021-05-16      ...  9.572148e-19  3.885298e-35  7.229485e-21  3.175978e-41   
2021-05-17      ...  4.786074e-19  1.942649e-35  3.614743e-21  1.587989e-41   
2021-05-18      ...  2.393037e-19  9.713246e-36  1.807371e-21  7.939944e-42   

Province_State                                                                 \
Admin2            Sweetwater         Teton     Uinta Unassigned      Washakie   
2021-05-16      3.299198e-06  1.013832e-28  0.000010  -0.501961  1.973288e-33   
2021-05-17      1.649599e-06  5.069159e-29  0.000005  -0.501961  9.866442e-34   
2021-05-18      8.247994e-07  2.534580e-29  0.000003  -0.501961  4.933221e-34   

Province_State                
Admin2                Weston  
2021-05-16      4.190824e-32  
2021-05-17      2.095412e-32  
2021-05-18      1.047706e-32  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
